In [27]:
## Importing Qiskit libraries
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
import numpy as np

In [28]:
## Settings
d = 1 # dimension
M = 2 # number of particles
L = 2 # number of qubits for momentum
N = L * d + d + 1 # nummber of qubits per one particle

In [29]:
def init(M): # initialize circuit
    particle_registers = []
    for i in range(M):
        particle_registers.append(QuantumRegister((L + 1) * d + 1, name = f'particle{i}'))
    particle_registers.append(QuantumRegister(L, name = f'aux')) # aux register for free Hamiltonian
    circuit = QuantumCircuit(*particle_registers)
    return circuit, particle_registers

In [23]:
#### examples of operation ####
## operate x gate to one certain qubit
# circuit.x(((L + 1) * d + 1) * M)
# circuit.h(particle_registers[1][0])

## operate x gate to one certain particle register
# circuit.x(particle_registers[0])

## operate x gate to all qubits
# for i in range(M):
#     circuit.x(particle_registers[i])

# circuit.draw('mpl') # 1つのparticleに対して下位 L* d ビットが運動量の大きさ、次のdビットが符号、最後1ビットがoccupancy

In [44]:
def setup_addition(circuit, reg1, reg2, reg3):
    # reg3にequal superpositionを生成
    # QuantumCircuitの1量子ビットゲートに対応するメソッド（circuit.hなど）に単一の量子ビットの代わりに
    # レジスタや量子ビットのリストを渡すと、含まれる全ての量子ビットに同じゲートをかけてくれる
    circuit.h(reg3)

    # 位相の単位（dphiの整数倍の位相をCPゲートでかけていく）
    dphi = 2. * np.pi / (2 ** reg3.size)

    # reg1とreg2それぞれの量子ビットで制御する
    for reg_ctrl in [reg1, reg2]:
        # 制御ビットに関するループ
        for ictrl, qctrl in enumerate(reg_ctrl):
            # reg3の標的ビットに関するループ
            for itarg, qtarg in enumerate(reg3):
                # C[P(phi)], phi = 2pi * 2^{ictrl} * 2^{itarg} / 2^{n3}
                circuit.cp(dphi * (2 ** (ictrl + itarg)), qctrl, qtarg)

    # 回路図を見やすくするためのバリア
    # circuit.barrier()

    # Inverse QFT
    for j in range(reg3.size // 2):
        circuit.swap(reg3[j], reg3[-1 - j])

    for itarg in range(reg3.size):
        for ictrl in range(itarg):
            power = ictrl - itarg - 1 + reg3.size
            circuit.cp(-dphi * (2 ** power), reg3[ictrl], reg3[itarg])
        
        circuit.h(reg3[itarg])
    return circuit
        
print('Defined function setup_addition')

Defined function setup_addition


In [82]:
def addition(circuit, reg_in, reg_out): # reg_in is the list of registers
    circuit.h(reg_out)
    dphi = 2. * np.pi / (2 ** reg_out.size)
    for reg_ctrl in reg_in:
        for ictrl, qctrl in enumerate(reg_ctrl):
            for itarg, qtarg in enumerate(reg_out):
                # C[P(phi)], phi = 2pi * 2^{ictrl} * 2^{itarg} / 2^{n3}
                circuit.cp(dphi * (2 ** (ictrl + itarg)), qctrl, qtarg)
    return circuit

In [119]:
enumerate([particle_registers[0][0], particle_registers[0][1], particle_registers[1][0], particle_registers[1][1], particle_registers[2][0], particle_registers[2][1]])
reg_in = [particle_registers[0][0], particle_registers[0][1], particle_registers[1][0], particle_registers[1][1], particle_registers[2][0], particle_registers[2][1]]
for reg_ctl in reg_in:
    x = 0

In [116]:
circuit, particle_registers = init(M) # initialization
reg_in = [particle_registers[0][:L], particle_registers[1][:L]]
reg_out = particle_registers[2]
add_circuit = addition(circuit, reg_in, reg_out)
add_gate = add_circuit.to_gate()
cadd_gate = add_gate.control(1)
circuit.append(cadd_gate, qargs=([particle_registers[0][0], particle_registers[0][1], particle_registers[1][0], particle_registers[1][1], particle_registers[2][0], particle_registers[2][1]]), cargs=(particle_registers[0][3]))
add_circuit.draw('mpl')

TypeError: 'Qubit' object is not iterable

In [60]:
# def make_addition_gate(M, L): # make addtition gate
#     add_circuit, add_particle_reg = init(M)
#     reg_in = []
#     for i in range(M):
#         reg_in.append(add_particle_reg[i][:L])
#     add_circuit = addition(add_circuit, reg_in, reg_out=add_particle_reg[M])
#     add_gate = add_circuit.to_gate()
#     # cadd_gate = add_gate.control(1)
#     return add_circuit

In [ ]:
# circuit, particle_registers = init(M)
# cadd_gate = make_addition_gate(M, L)
# circuit.append(cadd_gate, qargs=([particle_registers[0][:L] + particle_registers[1][:L] + particle_registers[2]]))
# # circuit.draw('mpl')